In [1]:
import glob
from math import sqrt, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from scipy.stats import norm

In [2]:
from brownian import brownian

In [3]:
# The number of dimensions of the space
d = 2
# The Wiener process parameter.
delta = 1
# Total time.
T = 2.0
# Number of steps.
N = 100
# Number of realizations to generate.
m = 50
# Set the image resolution
nx, ny = 50, 50

In [4]:
# Time step size
dt = T / N
# Create an empty array to store the realizations.
W = np.empty((d, m, N + 1))
# Initial values of x.
W[..., 0] = 0
# Set an array for time steps
t = np.linspace(0, N * dt, N + 1)
# Create a square grid
X, Y = np.linspace(0, 2 * pi, nx), np.linspace(0, 2 * pi, ny)

In [5]:
hF = lambda a, m, n: lambda x, y: a * np.sin(m * x) * np.sin(n * y)
sF = lambda a, m, n: lambda x, y: hF(a, m, n)(x, y).reshape((nx, ny, 1)) * np.exp(-(m + n) ** 2 * t / 2 / d).reshape((1, 1, *t.shape))

In [6]:
fourier = lambda F, c: lambda x, y: sum([sum([F(c[m, n], m+1, n+1)(x, y) for m in range(c.shape[0])]) for n in range(c.shape[1])])

In [7]:
np.random.seed(0)
c = np.random.randn(10, 10)

In [8]:
h = fourier(hF, c)
s = fourier(sF, c)

In [9]:
monte_carlo = np.empty((nx, ny, N+1))
for ix in range(nx):
    x = X[ix]
    for iy in range(ny):
        y = Y[iy]
        brownian(W[...,0], N, dt, delta, out=W[...,1:])
        monte_carlo[iy, ix, :] = np.mean(h(x + W[0], y + W[1]), axis=0)

In [10]:
analytic_solution = s(*np.meshgrid(X, Y))

In [11]:
name = 'fourier-2d-newp-lundi'
os.mkdir(f'images/gif/{name}')
for tPlot in range(0, N+1):
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(8, 5))
    fig.suptitle(f"Time: {dt*tPlot:0.2f}")
    im = ax1.imshow(monte_carlo[..., tPlot], vmin=0, vmax=1, cmap="coolwarm")
    ax1.set_title('Monte-Carlo approximation')
#     ax1.set_xticks([0, 0.5, 1], [0, 0.5, 1])
#     ax1.set_yticks([0, 0.5, 1], [0, 0.5, 1])
    im = ax2.imshow(analytic_solution[..., tPlot], vmin=0, vmax=1, cmap="coolwarm")
    ax2.set_title('Analytic solution')
#     ax1.set_xticks([0, 0.5, 1], [0, 0.5, 1])
#     ax1.set_yticks([0, 0.5, 1], [0, 0.5, 1])
    plt.tight_layout()
    fig.subplots_adjust(bottom=0.25)
    cbar_ax = fig.add_axes([0.1, 0.1, 0.8, 0.05])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation="horizontal")
    cbar.set_label('Heat')
    plt.savefig(f"images/gif/{name}/{tPlot:04.0f}.png", transparent=True)
    plt.close()

# Create the frames
frames = []
imgs = glob.glob(f"images/gif/{name}/*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

# Save into a GIF file that loops forever
frames[0].save(f'images/gif/{name}.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=1, loop=0)